In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-09-01'
final_date = '2022-09-30'
query_error_year = 2022
query_error_month = 9

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\09. SEPTIEMBRE\DATA\ERROR SEPTIEMBRE 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(102144, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_17480\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
92256,92256,3440043c-b3ea-4bc5-9b7a-ffbb9a6d3059,La Esquina Azul,KK2T83PFT,2022-09-01,"2,060.22",ERROR,Bank account information not available,2022-09-01,9,2022,NEW,SIN_CLABE,3440043c-b3ea-4bc5-9b7a-ffbb9a6d3059,2022-09-01,1,Actual
92170,92170,cefe197f-d074-41c5-9bb8-060cb129ad1d,Uniformes Rosy,DMW7HFMGN,2022-09-01,421.63,ERROR,Bank account information not available,2022-09-01,9,2022,NEW,SIN_CLABE,cefe197f-d074-41c5-9bb8-060cb129ad1d,2022-09-01,1,Actual
91134,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual
92171,92171,3a8b68bf-3cf0-45ce-ab24-2be8c8a4189c,la cuarenta y siete,6FTHLBNN4,2022-09-01,224.23,ERROR,DB,2022-09-01,9,2022,NEW,ERROR_USUAL,3a8b68bf-3cf0-45ce-ab24-2be8c8a4189c,2022-09-01,1,Actual
92173,92173,91b37b36-1069-43fa-91ce-e9dfa576adfe,Agustin,NHKHXFSY2,2022-09-01,295.14,ERROR,DB,2022-09-01,9,2022,NEW,ERROR_USUAL,91b37b36-1069-43fa-91ce-e9dfa576adfe,2022-09-01,1,Actual


In [9]:
query_error_current_month.shape

(10464, 17)

In [10]:
query_error_current_month['month'].unique()

array([9], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_17480\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
77261,77261,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,WGZ4PD6M8,2022-06-30,160.03,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN
48763,48763,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,VCSLG7L3X,2021-12-31,111.15,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN
58565,58565,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,Q3H87WHFZ,2022-03-11,143.74,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-08-01,8,2022,NEW,ERROR_USUAL,523288d9-dd39-4560-9dd0-c5a95a10df8a,2022-08-01,1,NaN
60897,60897,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,DMBZPNFZJ,2022-03-26,174.40,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN
61198,61198,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,KSL5GHG7G,2022-03-28,321.01,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-08-01,8,2022,NEW,ERROR_USUAL,523288d9-dd39-4560-9dd0-c5a95a10df8a,2022-08-01,1,NaN


In [15]:
query_new_last_month.shape

(32, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\09. SEPTEMBER\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\09. SEPTIEMBRE\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\08. AUGUST\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\08. AGOSTO\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [26]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\07. JULY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\07. JULIO\QUERIES\*.csv', recursive=False)

In [27]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [28]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [29]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
37230,DAILY,66e20d16-de56-464f-9f62-b5e1aeb1a809,Salon Stravaganzza,munozariadna40@gmail.com,2222QGPHF,2022-07-01,301.85,AZTECA,2022-07-01 10:47:59,Ariadna Muoz Martnez,'127832013353070294,2022-07-01 08:09:34,SANTANDER,"Ariadna Muoz Martnez,,301.85,12783201335307029...",SANT 2022-07-01.csv,2022-07-01,TRAD
6186,DAILY,11309c8f-72c8-49f8-bcf0-e319f8580fd7,Little Rome Pizza Suc San Ignacio,albertorafael_atc@hotmail.com,222QFQGRG,2022-07-01,130.32,"Banco Mercantil del Norte, S.A.",2022-07-01 10:28:28,Claudia Mnica Martinez De Los Santos,'072580011781091574,2022-07-01 08:04:56,SANTANDER,"Claudia Mnica Martinez De Los Santos,,130.32,0...",SANT 2022-07-01.csv,2022-07-01,TRAD
43548,DAILY,786e1a23-ba85-4bdd-98fe-c3da948e5a6c,CONSULTORIO MEDICO DRA. GIOVANA NALLELY RAMIRE...,dra.giovana.ramirez@gmail.com,2233LZZLR,2022-07-01,191.65,"Banco Mercantil del Norte, S.A.",2022-07-01 10:44:29,GIOVANA NALLELY RAMIREZ MONROY,'072760010667686976,2022-07-01 08:10:26,SANTANDER,"GIOVANA NALLELY RAMIREZ MONROY,,191.65,0727600...",SANT 2022-07-01.csv,2022-07-01,TRAD
4207,INSTANT,bbb368b4-8ed7-41cb-ad93-055ea8aaaa6e,Carniceria La Lluvia,ismaelsantiago735@gmail.com,2238CN3F5,2022-07-01,529.90,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-01 12:02:14,Ismael Santiago Vazquez,'659802020000376965,2022-07-01 11:55:20,SANTANDER,"Ismael Santiago Vazquez,,529.90,65980202000037...",FP-ALL 2022-07-01.csv,2022-07-01,FP
20625,DAILY,393a90ea-ccf0-4f37-8805-516515e90ee9,Buteco merendero pabellón,ulisesomar81@icloud.com,223K9NRFK,2022-07-01,356.47,"Scotiabank Inverlat, S.A.",2022-07-01 10:43:46,Jorge Eduardo Lopez Ochoa,'044010015050969792,2022-07-01 08:07:19,SANTANDER,"Jorge Eduardo Lopez Ochoa ,,356.47,04401001505...",SANT 2022-07-01.csv,2022-07-01,TRAD


In [30]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [31]:
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
37230,DAILY,66e20d16-de56-464f-9f62-b5e1aeb1a809,Salon Stravaganzza,munozariadna40@gmail.com,2222QGPHF,2022-07-01,301.85,AZTECA,2022-07-01 10:47:59,Ariadna Muoz Martnez,'127832013353070294,2022-07-01 08:09:34,SANTANDER,"Ariadna Muoz Martnez,,301.85,12783201335307029...",SANT 2022-07-01.csv,2022-07-01,TRAD,Pago del día
6186,DAILY,11309c8f-72c8-49f8-bcf0-e319f8580fd7,Little Rome Pizza Suc San Ignacio,albertorafael_atc@hotmail.com,222QFQGRG,2022-07-01,130.32,"Banco Mercantil del Norte, S.A.",2022-07-01 10:28:28,Claudia Mnica Martinez De Los Santos,'072580011781091574,2022-07-01 08:04:56,SANTANDER,"Claudia Mnica Martinez De Los Santos,,130.32,0...",SANT 2022-07-01.csv,2022-07-01,TRAD,Pago del día
43548,DAILY,786e1a23-ba85-4bdd-98fe-c3da948e5a6c,CONSULTORIO MEDICO DRA. GIOVANA NALLELY RAMIRE...,dra.giovana.ramirez@gmail.com,2233LZZLR,2022-07-01,191.65,"Banco Mercantil del Norte, S.A.",2022-07-01 10:44:29,GIOVANA NALLELY RAMIREZ MONROY,'072760010667686976,2022-07-01 08:10:26,SANTANDER,"GIOVANA NALLELY RAMIREZ MONROY,,191.65,0727600...",SANT 2022-07-01.csv,2022-07-01,TRAD,Pago del día
4207,INSTANT,bbb368b4-8ed7-41cb-ad93-055ea8aaaa6e,Carniceria La Lluvia,ismaelsantiago735@gmail.com,2238CN3F5,2022-07-01,529.90,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-01 12:02:14,Ismael Santiago Vazquez,'659802020000376965,2022-07-01 11:55:20,SANTANDER,"Ismael Santiago Vazquez,,529.90,65980202000037...",FP-ALL 2022-07-01.csv,2022-07-01,FP,Pago del día
20625,DAILY,393a90ea-ccf0-4f37-8805-516515e90ee9,Buteco merendero pabellón,ulisesomar81@icloud.com,223K9NRFK,2022-07-01,356.47,"Scotiabank Inverlat, S.A.",2022-07-01 10:43:46,Jorge Eduardo Lopez Ochoa,'044010015050969792,2022-07-01 08:07:19,SANTANDER,"Jorge Eduardo Lopez Ochoa ,,356.47,04401001505...",SANT 2022-07-01.csv,2022-07-01,TRAD,Pago del día


In [32]:
allfiles.shape

(8342790, 18)

# CRUCE 1 - MES ACTUAL

In [33]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,66e20d16-de56-464f-9f62-b5e1aeb1a809,Salon Stravaganzza,munozariadna40@gmail.com,2222QGPHF,2022-07-01,301.85,AZTECA,2022-07-01 10:47:59,Ariadna Muoz Martnez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,11309c8f-72c8-49f8-bcf0-e319f8580fd7,Little Rome Pizza Suc San Ignacio,albertorafael_atc@hotmail.com,222QFQGRG,2022-07-01,130.32,"Banco Mercantil del Norte, S.A.",2022-07-01 10:28:28,Claudia Mnica Martinez De Los Santos,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,786e1a23-ba85-4bdd-98fe-c3da948e5a6c,CONSULTORIO MEDICO DRA. GIOVANA NALLELY RAMIRE...,dra.giovana.ramirez@gmail.com,2233LZZLR,2022-07-01,191.65,"Banco Mercantil del Norte, S.A.",2022-07-01 10:44:29,GIOVANA NALLELY RAMIREZ MONROY,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,INSTANT,bbb368b4-8ed7-41cb-ad93-055ea8aaaa6e,Carniceria La Lluvia,ismaelsantiago735@gmail.com,2238CN3F5,2022-07-01,529.90,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-01 12:02:14,Ismael Santiago Vazquez,...,FP,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,393a90ea-ccf0-4f37-8805-516515e90ee9,Buteco merendero pabellón,ulisesomar81@icloud.com,223K9NRFK,2022-07-01,356.47,"Scotiabank Inverlat, S.A.",2022-07-01 10:43:46,Jorge Eduardo Lopez Ochoa,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [34]:
cruce_1.shape

(8342790, 26)

In [35]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
182,DAILY,36836a06-e282-49e6-88ed-aad1b2cc128d,Amigos protectores de los Animales A.C,info@fryda.org.mx,23HSVY9S5,2022-07-01,8.00,"Banco Santander, S.A.",2022-07-04 10:24:33,Amigos Protectores de los Animales A.C,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1658,DAILY,6dd027fb-4a0c-4e99-8a1f-9095200d087f,VIPort Lounge,viportlounge@gmail.com,2J4Z99MNQ,2022-07-01,"11,560.21","Banco Santander, S.A.",2022-07-01 10:48:38,Operadora Vport Lounges SC,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2253,DAILY,025d6e21-d470-4f71-b903-64da20b821c7,Lupita,letia.esra@gmail.com,2P8G4298S,2022-07-01,3.80,"Banco Azteca, S.A.",2022-07-01 10:27:11,Leticia Escobar Ramirez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2552,DAILY,55d01b24-af09-4f23-9d74-97b5d8891ef5,Los Artesanos,losartesanoscancun@gmail.com,2RVLKTSHQ,2022-06-24,"12,483.96","BBVA Bancomer, S.A.",2022-07-03 06:48:17,Jose Hector Herrera Fernandez,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2619,DAILY,4dd1ecab-4512-4660-babf-48c0527f21ae,Moonlight Spirits,vhra81@gmail.com,2SFBMY5VC,2022-07-01,918.33,"BBVA Bancomer, S.A.",2022-07-03 12:18:31,Distribuidora Onilikan de México S de R L de C V,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
duplicates_cruce_1_previo.shape

(202491, 26)

In [37]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_17480\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
1637606,INSTANT,faac4081-dcb1-49ff-90ec-375db4ba1086,Refaccionaria El Jaral,eric-negocio@hotmail.com,B3RDWTFDP,2022-07-19,292.26,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-19 16:24:57,Eric Guajardo Alvarado,...,FP,Pago del día,292.26,2022-09-05,2022-09-05,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4181413,INSTANT,d3bc09af-19b8-4fa0-8ce3-b35a5caf3f8d,Granos y semillas ROSITA,bautistaacevedom@gmail.com,QVJKYKHSY,2022-08-16,301.85,"Opciones Empresariales del Noreste, S.A. de C....",2022-08-16 12:07:12,Marcos Antonio Bautista Acevedo,...,FP,Pago del día,301.85,2022-09-07,2022-09-07,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4262675,INSTANT,7fc2e5dd-515b-4aea-b337-dcc444305479,Six,atrasquedolahuella@gmail.com,NBQJ77M58,2022-08-17,169.60,"Opciones Empresariales del Noreste, S.A. de C....",2022-08-17 16:24:40,Guadalupe Fontes Angel,...,FP,Pago del día,169.60,2022-09-05,2022-09-05,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4310440,INSTANT,7fc2e5dd-515b-4aea-b337-dcc444305479,Six,atrasquedolahuella@gmail.com,6PQYPGT84,2022-08-18,650.65,"Opciones Empresariales del Noreste, S.A. de C....",2022-08-18 20:52:26,Guadalupe Fontes Angel,...,FP,Pago del día,650.65,2022-09-05,2022-09-05,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4329195,INSTANT,7fc2e5dd-515b-4aea-b337-dcc444305479,Six,atrasquedolahuella@gmail.com,DPFSRY84W,2022-08-18,91.03,"Opciones Empresariales del Noreste, S.A. de C....",2022-08-18 14:14:29,Guadalupe Fontes Angel,...,FP,Pago del día,91.03,2022-09-05,2022-09-05,9.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW


In [38]:
cruce_1_final.shape

(4729, 26)

In [39]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [40]:
duplicates_cruce_1_final.shape

(0, 26)

# CRUCE 2 MES ACTUAL

In [41]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,92256,3440043c-b3ea-4bc5-9b7a-ffbb9a6d3059,La Esquina Azul,KK2T83PFT,2022-09-01,"2,060.22",ERROR,Bank account information not available,2022-09-01,9,2022,NEW,SIN_CLABE,3440043c-b3ea-4bc5-9b7a-ffbb9a6d3059,2022-09-01,1,Actual,Not found,NaN
1,92170,cefe197f-d074-41c5-9bb8-060cb129ad1d,Uniformes Rosy,DMW7HFMGN,2022-09-01,421.63,ERROR,Bank account information not available,2022-09-01,9,2022,NEW,SIN_CLABE,cefe197f-d074-41c5-9bb8-060cb129ad1d,2022-09-01,1,Actual,Not found,NaN
2,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Pago del día
3,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
4,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación


In [42]:
cruce_2.shape

(10989, 19)

In [43]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
2,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Pago del día
3,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
4,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
5,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
6,92171,3a8b68bf-3cf0-45ce-ab24-2be8c8a4189c,la cuarenta y siete,6FTHLBNN4,2022-09-01,224.23,ERROR,DB,2022-09-01,9,2022,NEW,ERROR_USUAL,3a8b68bf-3cf0-45ce-ab24-2be8c8a4189c,2022-09-01,1,Actual,TRAD,Pago del día


In [44]:
cruce_2_final.shape

(5254, 19)

In [45]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
2,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Pago del día
3,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
4,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
5,91134,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,SDPVNQ222,2022-08-29,"1,398.08",ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Reprogramación
236,85894,d617b557-4d5d-45ff-b8cc-4363645e2695,compubyte,WSZ9D9DQZ,2022-08-09,856.67,ERROR,DI,2022-09-01,9,2022,NEW,ERROR_USUAL,d617b557-4d5d-45ff-b8cc-4363645e2695,2022-09-01,1,Actual,TRAD,Pago del día


In [46]:
duplicates_cruce_2_final.shape

(917, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [47]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-09-01           0.00         242.43   
1                         2022-09-02           0.00      17,795.09   
2                         2022-09-03           0.00           0.00   
3                         2022-09-04           0.00           0.00   
4                         2022-09-05      32,671.41         432.41   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                       0.00           0.00    38,556.09       2,944.68   
1                       0.00           0.00    49,125.12      96,182.00   
2                       0.00           0.00    11,454.66           0.00   
3                   5,016.47           0.00    46,635.00       2,456.93   
4                  32,326.21           0.00    94,162.94      11,571.08   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0                41,743.20  
1               163,102.21  
2                11,454.66  
3                54,108.40  
4               171,164.05

In [48]:
group_tipo_pago.shape

(31, 8)

In [49]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-09-01         242.43    38,556.09       2,944.68   
1                       2022-09-02      17,795.09    49,125.12      96,182.00   
2                       2022-09-03           0.00    11,454.66           0.00   
3                       2022-09-04           0.00    51,651.47       2,456.93   
4                       2022-09-05      33,103.82   126,489.15      11,571.08   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0              41,743.20  
1             163,102.21  
2              11,454.66  
3              54,108.40  
4             171,164.05

In [50]:
group_esquema.shape

(31, 5)

# CRUCE 3 - MES ANTERIOR

In [51]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,DAILY,66e20d16-de56-464f-9f62-b5e1aeb1a809,Salon Stravaganzza,munozariadna40@gmail.com,2222QGPHF,2022-07-01,301.85,AZTECA,2022-07-01 10:47:59,Ariadna Muoz Martnez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,DAILY,11309c8f-72c8-49f8-bcf0-e319f8580fd7,Little Rome Pizza Suc San Ignacio,albertorafael_atc@hotmail.com,222QFQGRG,2022-07-01,130.32,"Banco Mercantil del Norte, S.A.",2022-07-01 10:28:28,Claudia Mnica Martinez De Los Santos,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,DAILY,786e1a23-ba85-4bdd-98fe-c3da948e5a6c,CONSULTORIO MEDICO DRA. GIOVANA NALLELY RAMIRE...,dra.giovana.ramirez@gmail.com,2233LZZLR,2022-07-01,191.65,"Banco Mercantil del Norte, S.A.",2022-07-01 10:44:29,GIOVANA NALLELY RAMIREZ MONROY,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,INSTANT,bbb368b4-8ed7-41cb-ad93-055ea8aaaa6e,Carniceria La Lluvia,ismaelsantiago735@gmail.com,2238CN3F5,2022-07-01,529.90,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-01 12:02:14,Ismael Santiago Vazquez,...,FP,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,DAILY,393a90ea-ccf0-4f37-8805-516515e90ee9,Buteco merendero pabellón,ulisesomar81@icloud.com,223K9NRFK,2022-07-01,356.47,"Scotiabank Inverlat, S.A.",2022-07-01 10:43:46,Jorge Eduardo Lopez Ochoa,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [52]:
cruce_3.shape

(8342790, 26)

In [53]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_17480\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2752442,DAILY,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,albertjosephbyermiller@gmail.com,KSL5GHG7G,2022-03-28,321.01,AZTECA,2022-08-01 06:24:11,Albert Joseph Byer Miller,...,TRAD,Reprogramación,321.01,2022-08-01,2022-08-01,8.00,ERROR,DEV.SPEICUENTA INEXISTENTE,ERROR_USUAL,NEW
2767164,DAILY,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,albertjosephbyermiller@gmail.com,Q3H87WHFZ,2022-03-11,143.74,AZTECA,2022-08-01 06:24:11,Albert Joseph Byer Miller,...,TRAD,Reprogramación,143.74,2022-08-01,2022-08-01,8.00,ERROR,DEV.SPEICUENTA INEXISTENTE,ERROR_USUAL,NEW
2824757,INSTANT,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,abarrotesrr4@gmail.com,DMBZPNFZJ,2022-03-26,174.40,"Banco Azteca, S.A.",2022-08-01 10:06:30,jesus emmanuel guevara diaz,...,FP,Reprogramación,174.40,2022-08-01,2022-08-01,8.00,ERROR,DH,ERROR_USUAL,NEW
2841332,INSTANT,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,abarrotesrr4@gmail.com,M9R2Z72FR,2022-03-28,115.94,"Banco Azteca, S.A.",2022-08-01 10:06:30,jesus emmanuel guevara diaz,...,FP,Reprogramación,115.94,2022-08-01,2022-08-01,8.00,ERROR,DH,ERROR_USUAL,NEW
2858830,INSTANT,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,abarrotesrr4@gmail.com,VCSLG7L3X,2021-12-31,111.15,"Banco Azteca, S.A.",2022-08-01 10:06:30,jesus emmanuel guevara diaz,...,FP,Reprogramación,111.15,2022-08-01,2022-08-01,8.00,ERROR,DH,ERROR_USUAL,NEW


In [54]:
cruce_3_final.shape

(32, 26)

In [55]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [56]:
duplicates_cruce_3_final.shape

(0, 26)

# CRUCE 4 - MES ANTERIOR

In [57]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,77261,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,WGZ4PD6M8,2022-06-30,160.03,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN,FP,Reprogramación
1,48763,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,VCSLG7L3X,2021-12-31,111.15,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN,FP,Reprogramación
2,58565,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,Q3H87WHFZ,2022-03-11,143.74,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-08-01,8,2022,NEW,ERROR_USUAL,523288d9-dd39-4560-9dd0-c5a95a10df8a,2022-08-01,1,NaN,TRAD,Reprogramación
3,60897,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,DMBZPNFZJ,2022-03-26,174.40,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN,FP,Reprogramación
4,61198,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,KSL5GHG7G,2022-03-28,321.01,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-08-01,8,2022,NEW,ERROR_USUAL,523288d9-dd39-4560-9dd0-c5a95a10df8a,2022-08-01,1,NaN,TRAD,Reprogramación


In [58]:
cruce_4.shape

(102, 19)

In [59]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,77261,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,WGZ4PD6M8,2022-06-30,160.03,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN,FP,Reprogramación
1,48763,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,VCSLG7L3X,2021-12-31,111.15,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN,FP,Reprogramación
2,58565,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,Q3H87WHFZ,2022-03-11,143.74,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-08-01,8,2022,NEW,ERROR_USUAL,523288d9-dd39-4560-9dd0-c5a95a10df8a,2022-08-01,1,NaN,TRAD,Reprogramación
3,60897,930b52f0-1c3e-473f-896b-62e4f659a5f6,Abarrotes La RR,DMBZPNFZJ,2022-03-26,174.40,ERROR,DH,2022-08-01,8,2022,NEW,ERROR_USUAL,930b52f0-1c3e-473f-896b-62e4f659a5f6,2022-08-01,1,NaN,FP,Reprogramación
4,61198,523288d9-dd39-4560-9dd0-c5a95a10df8a,De Aqui Y De Alla,KSL5GHG7G,2022-03-28,321.01,ERROR,DEV.SPEICUENTA INEXISTENTE,2022-08-01,8,2022,NEW,ERROR_USUAL,523288d9-dd39-4560-9dd0-c5a95a10df8a,2022-08-01,1,NaN,TRAD,Reprogramación


In [60]:
cruce_4_final.shape

(102, 19)

In [61]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
7,76854,9851621e-8d93-4b2f-acb7-9fec71e1eacb,Deposito El Guero,XXPNTCYJB,2022-06-29,161.46,ERROR,DM,2022-08-02,8,2022,NEW,ERROR_USUAL,9851621e-8d93-4b2f-acb7-9fec71e1eacb,2022-08-02,2,NaN,TRAD,Reprogramación
8,76854,9851621e-8d93-4b2f-acb7-9fec71e1eacb,Deposito El Guero,XXPNTCYJB,2022-06-29,161.46,ERROR,DM,2022-08-02,8,2022,NEW,ERROR_USUAL,9851621e-8d93-4b2f-acb7-9fec71e1eacb,2022-08-02,2,NaN,TRAD,Reprogramación
9,76854,9851621e-8d93-4b2f-acb7-9fec71e1eacb,Deposito El Guero,XXPNTCYJB,2022-06-29,161.46,ERROR,DM,2022-08-02,8,2022,NEW,ERROR_USUAL,9851621e-8d93-4b2f-acb7-9fec71e1eacb,2022-08-02,2,NaN,TRAD,Reprogramación
10,76541,9851621e-8d93-4b2f-acb7-9fec71e1eacb,Deposito El Guero,XXGX4LPLM,2022-06-27,"1,543.73",ERROR,DM,2022-08-02,8,2022,NEW,ERROR_USUAL,9851621e-8d93-4b2f-acb7-9fec71e1eacb,2022-08-02,2,NaN,TRAD,Reprogramación
11,76541,9851621e-8d93-4b2f-acb7-9fec71e1eacb,Deposito El Guero,XXGX4LPLM,2022-06-27,"1,543.73",ERROR,DM,2022-08-02,8,2022,NEW,ERROR_USUAL,9851621e-8d93-4b2f-acb7-9fec71e1eacb,2022-08-02,2,NaN,TRAD,Reprogramación


In [62]:
duplicates_cruce_4_final.shape

(95, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [63]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x                         
esquema_query                           ERROR_USUAL                      All
status_cubeta                                   NEW                         
tipo_dispersion                                  FP           TRAD          
tipo_pago                            Reprogramación Reprogramación          
0                         2022-08-01         744.06         464.75  1,208.81
1                         2022-08-02           0.00      19,328.59 19,328.59
2                         2022-08-15           0.00       3,629.82  3,629.82
3                         2022-08-24           0.00       7,430.19  7,430.19
4                                All         744.06      30,853.35 31,597.41

In [64]:
group_tipo_pago_last_month.shape

(5, 4)

In [65]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount           
esquema_query                         ERROR_USUAL        All
status_cubeta                                 NEW           
tipo_pago                          Reprogramación           
0                       2022-08-01       1,208.81   1,208.81
1                       2022-08-02      58,001.37  58,001.37
2                       2022-08-15      18,149.10  18,149.10
3                       2022-08-24      29,720.76  29,720.76
4                              All     107,080.04 107,080.04

In [66]:
group_esquema_last_month.shape

(5, 3)

In [67]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL SEPTIEMBRE 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()